In [12]:
# Iterates between different numbers of baffles and measures power flow
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy import constants as const
import poppy


fwhm_ = np.sqrt(np.log(2)/297.4178) * u.m 
ff = 500*u.GHz
ll = const.c/ff

wf = poppy.FresnelWavefront(beam_radius=fwhm_, wavelength=ll)

lenslet_aperture = poppy.GaussianAperture(fwhm=fwhm_)
wf *= lenslet_aperture

ring = poppy.CircularAperture(radius=0.05*u.m)
wf *= ring
y, x = wf.coordinates()
initial_power=np.sum(wf.intensity)

nbaff_power=np.zeros([5,1])
nbaff_spill=np.zeros([5,1])
for nbaff in range(1,6):
    wf_it=wf
    wf_it.propagate_fresnel(8.9238*u.cm+(5-nbaff)*u.cm)

    # up to 5 rings, each 2 cm apart
    for dummyvar in np.arange(nbaff,6):
        wf_it.propagate_fresnel(2*u.cm)
        wf_it *= ring
    
    wf_it.propagate_fresnel(14.1455*u.cm)

    temp = wf_it.intensity
    for m in np.arange(0,2048,dtype=int):
        for n in np.arange(0,2048,dtype=int):
            if (0.05)**2 < x[m,n]**2 + y[m,n]**2 :
                nbaff_spill[nbaff-1] += temp[m,n]  
            else:
                nbaff_power[nbaff-1] += temp[m,n]
    print('Transmitted power with '+str(nbaff)+' baffles:')
    print(str(100*nbaff_power[nbaff-1]/initial_power)+'%')
    print('Power spilled with '+str(nbaff)+' baffles:')
    print(str(100*nbaff_spill[nbaff-1]/initial_power)+'%')
    
#a=0 # Toggles zoom
#b=2048-a
#fig, ax = plt.subplots()
#c=ax.pcolormesh(x[a:b,a:b],y[a:b,a:b],z2[a:b,a:b])
#fig.colorbar(c,ax=ax)
#plt.show()

Transmitted power with 1 baffles:
[99.84364656]%
Power spilled with 1 baffles:
[0.03810914]%
Transmitted power with 2 baffles:
[99.74043157]%
Power spilled with 2 baffles:
[0.03809477]%
Transmitted power with 3 baffles:
[99.64143815]%
Power spilled with 3 baffles:
[0.04088208]%
Transmitted power with 4 baffles:
[99.53346823]%
Power spilled with 4 baffles:
[0.05705423]%
Transmitted power with 5 baffles:
[99.43154503]%
Power spilled with 5 baffles:
[0.05740808]%


1
2
3
4
